## What does our ZKP look like 

We have a bunch of stuff 

1. A witness which is information that lets us make the proof in our case this is knowledge of a variable x that satisfies our eqation. It also includes the intermediate values that are zkp uses. This is secret to the user they want to prove this information.
2. A set of gate constraints. Basically that all the multiplicaions and addition we do are correct. 
3. A copy constriant check
4. Input / output checks

# Plonk Tutorial
This tutorial is based upon https://www.vitalik.ca/general/2019/09/22/plonk.html it expands upon the ideas described there and teaches the user to build their own plonk implemenation in python.

This tutorial takes to forum of a series of challenges where the user eventually build a plonk implmentation of a single proof. 

Plonk allows us to make arbitary zero knoledge proofs. For the purposes of this tutorial we will prove that we know an x such that  $P(x) = x^3 + x + 5 = 35$ this is a toy problem

![](https://vitalik.ca/files/posts_files/plonk-files/Circuit.png)

You can see we have two kinds of constraints gate constraints and copy constraints. A constraint is like an assertion from python. The program can only continue running if this assertion is true.

We will first handle the gate constraints and then tackle the copy constraints. 

## Gen witness

So lets first find a satisfying solution to the probelm we are trying to make proofs about $x^3 + x + 5 = 35$

The variables a, b and c will be the checking of additions / multiplicaions operatirons. Where we define `a + b == c` or `a * b == c`

In [11]:
def gen_witness(x, y, z):
    a = [x, y, x*x, z]
    b = [x, y, y*y, z]
    c = [x*x, y*y, x*x+y*y, z*z]
    return(a, b, c)

Right so now that we have a, b , c we are ready to test. `is_satisfied` tests that our witness matches the constraints we are planing to add. 

Basically a[0] * b[0] = c[0] check this is a multiplicaion
a[2] + b[2] = c[2] check this is an addition.

In [12]:
def is_satisfied_witness(a, b, c):
    assert(a[0] * b[0] == c[0])
    assert(a[1] * b[1] == c[1])
    assert(a[2] + b[2] == c[2])
    assert(a[3] * b[3] == c[3])

In [13]:
# from plonk.sample_problem import gen_witness, is_satisfied_witness

# a, b , c = gen_witness(1)

# Uncomment the next line and run
# is_satisfied_witness(a,b,c)

Reader should investigate why this fails and why the next passes

In [14]:
a, b, c = gen_witness(3, 4, 5)
is_satisfied_witness(a, b, c)

## In plonk everything is a polynomial

In the previous section we generated our witness. A witness is a valid solution to our constraints. Where here our constraints are $x^3 + x + 5 == 35$

Next we want to define the actual constraints. They will be defined as a polynomial. Lets start out by creating a function eval_poly which takes a polynomial and evaluates it at a given point. Take the polynomial $1 + x + x^2 = y$ which is defined by this list [1,1,1] from lowest degree (ie starting at the $1*x^0$ ) to highest (1*x^2) 

In [15]:
## User input here. 
def eval_poly(coef, x):
    res = []
    power = 1
    for i in coef:
        res.append(i * power)
        power = power * x
    return(round(sum(res)))

In [16]:
assert(eval_poly([1,1,1], 2) == 7 )
assert(eval_poly([-2, 7, -5, 1], 0) == -2)
assert(eval_poly([-2, 7, -5, 1], 1) == 1)
assert(eval_poly([-2, 7, -5, 1], 2) == 0)
assert(eval_poly([-2, 7, -5, 1], 3) == 1)

Okay now it seems out polynomial evaluations are working :)

Our mul / add constraint is defined by $\left(Q_{L_{i}}\right) a_{i}+\left(Q_{R_{i}}\right) b_{i}+\left(Q_{O_{i}}\right) c_{i}+\left(Q_{M_{i}}\right) a_{i} b_{i}+Q_{C_{i}}=0$ we can use this to check additions and multiplications. Define the constraint polynomial.


In [17]:
def constraint_polynomial(Qli, Qri, Qmi, Qoi, Qci, ai, bi, ci):
    return(Qli*ai + Qri*bi + Qoi*ci + Qmi*ai*bi + Qci == 0)

def validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c):
    for Qli,Qri,Qmi,Qoi,Qci,ai,bi,ci in zip (Ql,Qr,Qm,Qo,Qc,a,b,c):
        if (constraint_polynomial(Qli,Qri,Qmi,Qoi,Qci,ai,bi,ci) == False):
            return(False)
    return(True)

In [18]:
def test_addition():
    # constraints
    Ql = [1]
    Qr = [1]
    Qm = [0]
    Qo = [-1]
    Qc = [0]

    # witness
    a = [1]
    b = [1]
    c = [2]

    assert ( validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c) == True)


def test_mul():

    # constraints
    Ql = [0]
    Qr = [0]
    Qm = [1]
    Qo = [-1]
    Qc = [0]

    # witness
    a = [1]
    b = [1]
    c = [1]

    assert ( validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c) == True)

    

def test_constant():
    # constraints
    Ql = [1]
    Qr = [1]
    Qm = [0]
    Qo = [0]
    Qc = [-10]

    # witness
    a = [10]
    b = [0]
    c = [10]

    assert ( validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c) == True)
    
test_addition()
test_mul()
test_constant()



Okay so now we are doing multiplicaions and additions. We can validate manually that all of these are being done correctly. Right so this is working we can make constraints. So lets make all the constraints for our system. First lets make some helpers that drop the constraints where we need then make sure they pass the tests :)

In [19]:
from plonk.constraint import add_add_constarint, add_mul_constarint, add_constant_constraint

Okay now lets add the actual constraints. By setting Ql, Qr, Qm , Qo and Qc such that it evaluates to a multipicion constaint at Ql[0] and an addition at Ql[2]. 

In [20]:
def gen_constraints():
    # Prove that I know an X such that X*x*x + x +5 == 35

    # init constraints 
    Ql = []
    Qr = []
    Qm = []
    Qo = []
    Qc = []

    # set constraints 
    Ql, Qr, Qm, Qo, Qc = add_mul_constarint(Ql, Qr, Qm, Qo, Qc)
    Ql, Qr, Qm, Qo, Qc = add_mul_constarint(Ql, Qr, Qm, Qo, Qc)
    Ql, Qr, Qm, Qo, Qc = add_add_constarint(Ql, Qr, Qm, Qo, Qc)
    Ql, Qr, Qm, Qo, Qc = add_mul_constraint(Ql, Qr, Qm, Qo, Qc)
    return(Ql, Qr, Qm, Qo, Qc)

## Copy constraints 
So at the moment the system is not secure. Basically we are checking that the variables at location 

1. `a[0] * b[0] == c[0]` 
2. `a[1] * b[1] == c[1]`
3. `a[2] + b[2] == c[2]`


But we are just hoping that `a[1] == c[0]` we need to add constraints to make sure that we copy c[0] to a[1] these are called copy constraints you may also have heard of them refered to as permutation arguments. 

TODO: Do the actual attack

The naive thing to do is to do these checks manually. Basically make sure that each variable is equal to the other. The probelm with this is that it means that we need to check every variable which breaks privacy and succintness. Instead we will find a way to do this check using polynomials. 

Right now we have our witness which is `witness = a + b + c` and we want to prove that the value at `witness[0] == witness[8] == witness[7] == witness[6]` all of these values corresponding to our initial x. 

So now we need to make 3 polynomials the first to return the index of witness we want to look up, `witness_x_1`. The second `witness_x_2` to return the index after the permutation has been applied. And the third witness_y which returns the actual value of that witness at a given index. 

Write code that returns all of these. 

hint: use `from scipy.interpolate import lagrange` to interpolate a polynomial that passes through several points. 

In [30]:
from scipy.interpolate import lagrange
import numpy as np

def find_permutation(witness):
    domain = list(range(len(witness)))
    permutation = np.random.permutation(domain)
    return lagrange(domain, permutation).coef[::-1]

witness = [3, 4, 9, 16, 25, 34, 12, 12, 25]

witness_x_1 = find_permutation(witness)
witness_x_2 = find_permutation(witness)
witness_y = lagrange(list(range(len(witness))), witness).coef[::-1]

def polynomial_eval(poly, x):
    return sum(c * (x**i) for i, c in enumerate(poly))

# assetion
for i, val in enumerate(witness):
    assert(val == polynomial_eval(witness_y, i))

ModuleNotFoundError: No module named 'scipy'

The test code below checks that witness_y returns the same results when we use the permutated indexes or the non permuated version. This means that each value there matches. 

# converted this into markdown instead
```py
# Okay now lets rearrange it so that the values get swapped when they match
from plonk.sample_problem import gen_copy_constraints

witness_x_a_perm, witness_x_b_perm, witness_x_c_perm, copy_constraints = gen_copy_constraints()

for i in range(0,len(a)):
    assert(polynomial_eval(witness_y , polynomial_eval(witness_x_a, i)) == 
           polynomial_eval (witness_y ,polynomial_eval(witness_x_a_perm, i)))

for i in range(len(a), len(a)*2):
    assert(polynomial_eval(witness_y , polynomial_eval(witness_x_b, i)) == 
           polynomial_eval (witness_y ,polynomial_eval(witness_x_b_perm, i)))
    
for i in range(len(a)*2, len(a)*3):
    assert(polynomial_eval(witness_y , polynomial_eval(witness_x_c, i)) == 
           polynomial_eval (witness_y ,polynomial_eval(witness_x_c_perm, i)))
```

So now we have a way of checking permutations with polynomials. But we still need to check every variable which means we have not really gained anything. So next we will embed these three polynomials in a third such that we can check batches of permutations at once. 

To do this we take a random linear combination of witness_x_1 and witness_y. $rlc = v1 + wintess_x_1 + v2*witness_y$

Then we calculate P(x) where P(0) = 1 and $P(x+1) = p(x)* rlc$

Then we do the same for witness_x_2 calculating P_2(x). Because v1 and v2 are random numbers we know that P_1(i) == P_2(i) if and only if witness_Y gives the same results when evaluated on witness_x_1(0:i) and witness_x_2(0:i)

In [18]:
from plonk.copy_constraint import copy_constraint_simple 

# we have to generate v1 and v2 after a, b and c have been fixed.
v1 = hash(str(a + b + c))
v2 = hash(str(c + a + b))

eval_domain = range(0, len(a)*3)


# x, Y , Px_a, rlc = copy_constraint_simple(range(0,len(a)), witness_x_a, witness_y, v1, v2)
# x, Y , Px_b, rlc = copy_constraint_simple(range(len(a),len(a)*2), witness_x_b, witness_y, v1, v2)
# x, Y , Px_c, rlc = copy_constraint_simple(range(len(a)*2,len(a)*3), witness_x_c, witness_y, v1, v2)
x, Y , Px_a, rlc = copy_constraint_simple(range(0,len(a)), witness_x_1, witness_y, v1, v2)
x, Y , Px_b, rlc = copy_constraint_simple(range(len(a),len(a)*2), witness_x_2, witness_y, v1, v2)
x, Y , Px_c, rlc = copy_constraint_simple(range(len(a)*2,len(a)*3), witness_x_2, witness_y, v1, v2)

# calcualte permutated polynomial    
# x_1, Y_1 , Px_a_prime, rlc_1 = copy_constraint_simple(range(0,len(a)), witness_x_a_perm, witness_y, v1, v2)
# x_1, Y_1 , Px_b_prime, rlc_1 = copy_constraint_simple(range(len(a),len(a)*2), witness_x_b_perm, witness_y, v1, v2)
# x_1, Y_1 , Px_c_prime, rlc_1 = copy_constraint_simple(range(len(a)*2,len(a)*3), witness_x_c_perm, witness_y, v1, v2)
x_1, Y_1 , Px_a_prime, rlc_1 = copy_constraint_simple(range(0,len(a)), witness_x_1, witness_y, v1, v2)
x_1, Y_1 , Px_b_prime, rlc_1 = copy_constraint_simple(range(len(a),len(a)*2), witness_x_2, witness_y, v1, v2)
x_1, Y_1 , Px_c_prime, rlc_1 = copy_constraint_simple(range(len(a)*2,len(a)*3), witness_x_2, witness_y, v1, v2)

assert(Px_a[-1] * Px_b[-1] * Px_c[-1] == Px_a_prime[-1] * Px_b_prime[-1] * Px_c_prime[-1])
assert(Px_a[0] == Px_b[0] == Px_c[0] == Px_a_prime[0] == Px_b_prime[0] == Px_c_prime[0] == 1 )

So now we can evaluate many copy constraints by simply checking a single point. But the problem is that the verifier needs to compute the Px_a ... Px_c_prime. We want to come up with a way so that they don't need to evaluate these instead letting the prover produce an argument that they have evaluated them correctly and minimize the verifiers work. We will do that in after the next section. In the next section we will make a quick fft sidetrack cos we need that to make a performant prover. 


## Part x: FFT

WILL NOT CONTINUE FROM HERE AS NO LONGER PART OF ASSIGNMEMNT, content below is removed!